In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow_hub as hub
import time
from tqdm import tqdm
import math
import json

In [6]:
path = os.getcwd()
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed = hub.load(path+'\\model')

Vectorisation

In [3]:
df = pickle.load(open(path+'\\cleaned_data.sav', 'rb'))

In [5]:
sentences = np.array(df['abstract'])

In [6]:
abstract_vec = np.zeros(shape=(1, 512))
groups_num = 20 # depends on GPU memery (8G set to 20 works fine for me)
for i in tqdm(range(math.ceil(len(sentences)/groups_num))):
    if (i < math.ceil(len(sentences)/groups_num)-1):
        vec = sentences[(i*groups_num):((i+1)*groups_num)]
    else:
        vec = sentences[(i*groups_num):len(sentences)]
    embed_vectors = embed(vec).numpy()
    abstract_vec = np.append(abstract_vec, embed_vectors, axis=0)
    
abstract_vec = np.delete(abstract_vec, 0, axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 8026/8026 [2:50:42<00:00,  1.28s/it]


In [8]:
abstract_qa = sentences, abstract_vec
pickle.dump(abstract_qa, open('./tu.pkl', 'wb'))

Model

In [3]:
abstract_sent, abstract_vec = pickle.load(open('./abstract_qa.pkl', 'rb'))

In [4]:
def query(text):
    corr = np.inner(embed([text]).numpy(), abstract_vec)[0]
    return abstract_sent[np.argmax(corr)]

In [7]:
query('What is the best covid treatment?')

'Apart from the treatment options, protocols and preventive measures against COVID, an essential step prior to the treatment is its quick and accurate diagnosis. There are several diagnostic protocols prevailing currently for the identification of COVID infection. The Gold standard available diagnosis is based on the identification of pathogen by RTPCR analysis . This involves extraction of the genetic material of the virus followed by its amplification using specific primers and identification.'

Get top k

In [20]:
corr = np.inner(embed(['What is the best covid treatment?']).numpy(), abstract_vec)[0]

In [21]:
corr

array([ 0.05221304,  0.04399603, -0.00804991, ...,  0.01596651,
       -0.02183577,  0.10824992])

In [22]:
#top 1
np.argmax(corr)

20528

In [17]:
def topk(k):
    return (-corr).argsort()[:k]

In [24]:
#top 2
topk(2)

array([20528, 34667], dtype=int64)